Adding county columns and state moved to column

In [1]:
import pandas as pd
import numpy as np
import pickle

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# load data
pklFile = "/content/drive/MyDrive/W209_final_project/Data/PickleFiles/df_final20230221.pkl"
df = pd.read_pickle(pklFile)

In [3]:
df = df.drop(axis = 1, labels = ["state", "county", "STATE", "STATENS", "STUSAB"])

In [4]:
df[df["STATE_NAME"] == "Arizona"]

,MOVEDIN,GEOID1,GEOID2,MOVEDOUT,FULL1_NAME,FULL2_NAME,MOVEDNET,YEAR,STATE_NAME
11504,64,04001,02020,0,"Apache County, Arizona","Anchorage Municipality, Alaska",64,2010,Arizona
11505,36,04001,02090,4,"Apache County, Arizona","Fairbanks North Star Borough, Alaska",32,2010,Arizona
11506,0,04001,02122,12,"Apache County, Arizona","Kenai Peninsula Borough, Alaska",-12,2010,Arizona
11507,0,04001,04003,166,"Apache County, Arizona","Cochise County, Arizona",-166,2010,Arizona
11508,177,04001,04005,94,"Apache County, Arizona","Coconino County, Arizona",83,2010,Arizona
...,...,...,...,...,...,...,...,...,...
4984829,0,04027,5503968175,4,"Yuma County, Arizona","Ripon City, Fond du Lac County, Wisconsin",-4,2020,Arizona
4984830,8,04027,5505578200,0,"Yuma County, Arizona","Sullivan Town, Jefferson County, Wisconsin",8,2020,Arizona
4984831,41,04027,5505939225,0,"Yuma County, Arizona","Kenosha City, Kenosha County, Wisconsin",41,2020,Arizona
4984832,76,04027,5506127250,0,"Yuma County, Arizona","Franklin Town, Kewaunee County, Wisconsin",76,2020,Arizona


In [5]:
df.head()

,MOVEDIN,GEOID1,GEOID2,MOVEDOUT,FULL1_NAME,FULL2_NAME,MOVEDNET,YEAR,STATE_NAME
0,38,01001,01003,11,"Autauga County, Alabama","Baldwin County, Alabama",27,2010,Alabama
1,0,01001,01005,15,"Autauga County, Alabama","Barbour County, Alabama",-15,2010,Alabama
2,44,01001,01007,0,"Autauga County, Alabama","Bibb County, Alabama",44,2010,Alabama
3,98,01001,01009,59,"Autauga County, Alabama","Blount County, Alabama",39,2010,Alabama
4,12,01001,01013,0,"Autauga County, Alabama","Butler County, Alabama",12,2010,Alabama


IDENTIFIERS REFERENCE: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

In [6]:
# get everything to 5 digit code
df["GEOID1"] = df["GEOID1"].str[:2] + df["GEOID1"].str[2:5]
df["GEOID2"] = df["GEOID2"].str[:2] + df["GEOID2"].str[2:5]

In [7]:
# Add human readable labels
dfCounty2010_2014 = pd.read_csv("https://www2.census.gov/geo/docs/reference/codes/national_county.txt", sep=",", dtype=object) # 2010 - 2014 encodings
dfCounty2010_2014["GEOID"] = dfCounty2010_2014["State ANSI"] + dfCounty2010_2014["County ANSI"]
dfCounty2010_2014 = dfCounty2010_2014.drop(["ANSI Cl"], axis = 1)

dfCounty2020 = pd.read_csv("https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt", sep="|", dtype=object) # 2015 - 2020 encodings
dfCounty2020["GEOID"] = dfCounty2020['STATEFP'] + dfCounty2020["COUNTYFP"]
dfCounty2020 = dfCounty2020.drop(["CLASSFP", "FUNCSTAT", 'COUNTYNS'], axis = 1)

In [8]:
dfCounty = pd.merge(dfCounty2010_2014, dfCounty2020, how = "outer", on = "GEOID")
dfCounty.head()

,State,State ANSI,County ANSI,County Name,GEOID,STATE,STATEFP,COUNTYFP,COUNTYNAME
0,AL,01,001,Autauga County,01001,AL,01,001,Autauga County
1,AL,01,003,Baldwin County,01003,AL,01,003,Baldwin County
2,AL,01,005,Barbour County,01005,AL,01,005,Barbour County
3,AL,01,007,Bibb County,01007,AL,01,007,Bibb County
4,AL,01,009,Blount County,01009,AL,01,009,Blount County


In [9]:
# restructure
dfCounty.isnull().sum()

State          4
State ANSI     4
County ANSI    4
County Name    4
GEOID          0
STATE          4
STATEFP        4
COUNTYFP       4
COUNTYNAME     4
dtype: int64

In [10]:
# fill in empty
for i, k in zip(list(dfCounty2010_2014.columns), list(dfCounty2020.columns)): 
  dfCounty[k] = dfCounty[k].fillna(dfCounty[i])

del dfCounty2010_2014, dfCounty2020

In [11]:
# quality check
dfCounty.isnull().sum()

State          4
State ANSI     4
County ANSI    4
County Name    4
GEOID          0
STATE          0
STATEFP        0
COUNTYFP       0
COUNTYNAME     0
dtype: int64

In [12]:
# remove extra columns
dfCounty.drop(axis = 1, inplace = True, labels = ['State', 'State ANSI', 'County ANSI', 'County Name', "STATEFP", "COUNTYFP"])

In [13]:
dfCounty.head()

,GEOID,STATE,COUNTYNAME
0,01001,AL,Autauga County
1,01003,AL,Baldwin County
2,01005,AL,Barbour County
3,01007,AL,Bibb County
4,01009,AL,Blount County


In [14]:
df.head()

,MOVEDIN,GEOID1,GEOID2,MOVEDOUT,FULL1_NAME,FULL2_NAME,MOVEDNET,YEAR,STATE_NAME
0,38,01001,01003,11,"Autauga County, Alabama","Baldwin County, Alabama",27,2010,Alabama
1,0,01001,01005,15,"Autauga County, Alabama","Barbour County, Alabama",-15,2010,Alabama
2,44,01001,01007,0,"Autauga County, Alabama","Bibb County, Alabama",44,2010,Alabama
3,98,01001,01009,59,"Autauga County, Alabama","Blount County, Alabama",39,2010,Alabama
4,12,01001,01013,0,"Autauga County, Alabama","Butler County, Alabama",12,2010,Alabama


In [15]:
df.shape

(5437049, 9)

In [16]:
df = pd.merge(left= df, right = dfCounty, left_on = "GEOID1", right_on = "GEOID", suffixes = ("_OG1", "_FROM"), how = "outer")
df = pd.merge(left= df, right = dfCounty, left_on = "GEOID2", right_on = "GEOID", suffixes = ("_FROM", "_TO"), how = "outer")
df.isnull().sum()

MOVEDIN            28
GEOID1             28
GEOID2             28
MOVEDOUT           28
FULL1_NAME         28
FULL2_NAME         28
MOVEDNET           28
YEAR               28
STATE_NAME         28
GEOID_FROM         14
STATE_FROM         14
COUNTYNAME_FROM    14
GEOID_TO           14
STATE_TO           14
COUNTYNAME_TO      14
dtype: int64

In [17]:
df.shape

(5437077, 15)

In [18]:
# cleanup columns
df.drop(inplace = True, axis = 1, labels = ["GEOID1", "GEOID2", "FULL1_NAME", "FULL2_NAME"])

In [19]:
df.head()

,MOVEDIN,MOVEDOUT,MOVEDNET,YEAR,STATE_NAME,GEOID_FROM,STATE_FROM,COUNTYNAME_FROM,GEOID_TO,STATE_TO,COUNTYNAME_TO
0,38,11,27.0,2010.0,Alabama,01001,AL,Autauga County,01003,AL,Baldwin County
1,78,7,71.0,2011.0,Alabama,01001,AL,Autauga County,01003,AL,Baldwin County
2,119,27,92.0,2012.0,Alabama,01001,AL,Autauga County,01003,AL,Baldwin County
3,116,62,54.0,2013.0,Alabama,01001,AL,Autauga County,01003,AL,Baldwin County
4,77,88,-11.0,2014.0,Alabama,01001,AL,Autauga County,01003,AL,Baldwin County


In [20]:
# add rustbelt label
rustbelt_states = ['Illinois', 'Indiana', 'Michigan', 'Missouri', 'New York', 'Ohio', 'Pennsylvania', 'West Virginia', 'Wisconsin']

states = pd.read_csv("/content/drive/MyDrive/W209_final_project/Data/state_abbreviations.csv")

In [21]:
df["STATE_FROM"] = pd.merge(df["STATE_FROM"], states, left_on = "STATE_FROM", right_on = "Abbreviation", how = "outer")["STATE"]
df["STATE_TO"] = pd.merge(df["STATE_TO"], states, left_on = "STATE_TO", right_on = "Abbreviation", how = "outer")["STATE"]

In [22]:
df.drop(inplace = True, axis = 1, labels = ['STATE_NAME'])
df.head()

,MOVEDIN,MOVEDOUT,MOVEDNET,YEAR,GEOID_FROM,STATE_FROM,COUNTYNAME_FROM,GEOID_TO,STATE_TO,COUNTYNAME_TO
0,38,11,27.0,2010.0,01001,Alabama,Autauga County,01003,Alabama,Baldwin County
1,78,7,71.0,2011.0,01001,Alabama,Autauga County,01003,Alabama,Baldwin County
2,119,27,92.0,2012.0,01001,Alabama,Autauga County,01003,Alabama,Baldwin County
3,116,62,54.0,2013.0,01001,Alabama,Autauga County,01003,Alabama,Baldwin County
4,77,88,-11.0,2014.0,01001,Alabama,Autauga County,01003,Alabama,Baldwin County


In [23]:
df.isnull().sum()

MOVEDIN            28
MOVEDOUT           28
MOVEDNET           28
YEAR               28
GEOID_FROM         14
STATE_FROM         15
COUNTYNAME_FROM    14
GEOID_TO           14
STATE_TO           15
COUNTYNAME_TO      14
dtype: int64

In [24]:
def rustbelt_state_label(s): 
  if s in rustbelt_states: 
    return "Yes"
  else:
    return "No"

In [25]:
df["STATE_FROM_RustBelt_Label"] = df['STATE_FROM'].apply(rustbelt_state_label)
df["STATE_TO_RustBelt_Label"] = df['STATE_TO'].apply(rustbelt_state_label)

In [ ]:
# Remove Null columns


In [26]:
import datetime
today = datetime.date.today()
today = datetime.date.strftime(today, "%Y%m%d")

In [27]:
# Write out csv
df.to_csv("/content/drive/MyDrive/W209_final_project/Data/df_county_sep_"+today+".csv")

In [29]:
df[df["STATE_TO"] == "Washington"]

,MOVEDIN,MOVEDOUT,MOVEDNET,YEAR,GEOID_FROM,STATE_FROM,COUNTYNAME_FROM,GEOID_TO,STATE_TO,COUNTYNAME_TO,STATE_FROM_RustBelt_Label,STATE_TO_RustBelt_Label
3852732,11,0,11.0,2012.0,12109,Oregon,St. Johns County,44003,Washington,Kent County,No,No
3852733,0,16,-16.0,2012.0,12109,Oregon,St. Johns County,44003,Washington,Kent County,No,No
3852734,69,50,19.0,2013.0,12109,Oregon,St. Johns County,44003,Washington,Kent County,No,No
3852735,0,30,-30.0,2013.0,12109,Oregon,St. Johns County,44003,Washington,Kent County,No,No
3852736,69,0,69.0,2013.0,12109,Oregon,St. Johns County,44003,Washington,Kent County,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...
3933639,4,0,4.0,2013.0,55129,Pennsylvania,Washburn County,55133,Washington,Waukesha County,Yes,No
3933640,4,0,4.0,2014.0,55129,Pennsylvania,Washburn County,55133,Washington,Waukesha County,Yes,No
3933641,4,0,4.0,2015.0,55129,Pennsylvania,Washburn County,55133,Washington,Waukesha County,Yes,No
3933642,5,0,5.0,2016.0,55129,Pennsylvania,Washburn County,55133,Washington,Waukesha County,Yes,No
